In [28]:
import numpy as np

In [1]:
from datasets import load_dataset
dataset = load_dataset("squad_v2")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [2]:
dft = dataset['train']

In [45]:
lengths_context = [len(x) for x in dft['context']]
lengths_question = [len(y) for y in dft['question']]
lengths_answer = [len(x['text'][0]) if x['text'] else 0  for x in dft['answers']]

In [46]:
avg_context = sum(lengths_context) / len(lengths_context)
min_context = min(lengths_context)
max_context = max(lengths_context)
print(f'Context length: avg: {avg_context}, min: {min_context}, max: {max_context}')

Context length: avg=754.5662873410631, min=151, max=3706


In [48]:
avg_question = sum(lengths_question) / len(lengths_question)
min_question = min(lengths_question)
max_question = max(lengths_question)
print("Question length: avg: {}, min: {}, max: {}".format(avg_question, min_question, max_question))

Question length: avg: 58.50773870272178, min: 1, max: 25651


In [40]:
avg_answer = sum(lengths_answer) / len(lengths_answer)
min_answer = min(lengths_answer)
max_answer = max(lengths_answer)
print("Answer length: avg: {}, min: {}, max: {}".format(avg_answer, min_answer, max_answer))

Answer length: avg: 13.423760157766711, min: 0, max: 239


In [49]:
from transformers import AutoTokenizer, RobertaForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained('deepset/roberta-base-squad2')
model = RobertaForQuestionAnswering.from_pretrained('deepset/roberta-base-squad2')

C:\Users\kilan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kilan\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
